## Parte 1: Modelagem Conceitual 

### 1.1. Diagrama ER

In [1]:
from IPython.display import Image, display

# URL da imagem
url = "https://i.postimg.cc/nz0mBWkS/der-tp1-BDA-drawio.png"

# Exibir a imagem
display(Image(url=url))

### 1.2. Decisões de modelagem
- Relacionameto  **CLIENTE - PEDIDO: 1:N**\
Este relacionamento é 1:N, pois um cliente pode realizar diversos pedidos, entretanto, um pedido só pode estar vinculado a um único cliente.\
Além disso, um cliente pode nao realizar nenhum pedido, mas todo pedido deve estar associado a um cliente.

- Relacionamento **PEDIDO - PRODUTO: N:M**\
Este relacionamento é N:M, pois um pedido pode conter diversos produtos e um tipo de produto pode estar em diversos pedidos diferentes.\
Além disso, todo pedido deve conter produtos, mas nem todo produto precisa estar em um pedido.

- Relacionamento **PRODUTO - CARGA: N:M**\
Foi-se considerado como carga o conjunto de itens comprados. Exemplo: comprou-se os itens a, b e c, logo a carga será o pacote com os itens do pedido.  
Este relacionamento é N:M, pois um mesmo tipo de produto pode estar em diversas cargas diferentes, e uma mesma carga pode conter vários tipos de produtos diferentes.
Além disso, toda carga deve conter produtos, mas nem todo produto precisa estar em uma carga.

- Relacionamento **CLIENTE - ENDEREÇO: N:1**\
Este relacionamento é N:1, pois um cliente , mas em um mesmo endereço pode residir diversos clientes.
Além disso, todo cliente deve residir em um endereço, mas nem todo endereço necessita ter um cliente.

- Relacionamento **CARGA - VEÍCULO: N:1**\
Este relacionamento é N:1, pois uma carga é transportada por apenas um veículo, mas um veículo pode transportar várias cargas.
Além disso, toda carga deve ser transportada por um veículo, mas nem todo veículo precisa transportar uma carga.

- Relacionamento **VEÍCULO - MOTORISTA: 1:1**:\
Este relacionamento é 1:1, pois um veículo está associado a apenas um motorista e um motorista está associado a apenas o seu único veículo.
Além disso, um veículo pode não estar associado a nenhum motorista, e podem haver motoristas não associados a nenhum veículo.

- Relacionamento **MOTORISTA - ROTAS: 1:N**:\
Este relacionamento é 1:N, pois um mesmo motorista pode realizar múltiplas rotas diferentes, mas uma rota está associada a um único motorista.
Além disso, este é um relacionamento fraco, uma vez que se motorista for removido do banco de dados, a rota não irá existir pois não haverá ninguém para realizá-la.

- Relacionamto **ENDEREÇO - CARGA: 1:N (Tem origem):**\
Este relacionamento é 1:N, pois um mesmo endereço pode ser origem de várias cargas diferentes, mas uma carga específica só possui um endereço de origem.
Além disso, toda carga necessariamente tem origem em um endereço, mas nem todo endereço precisa ser a origem de uma carga.

- Relacionamto **ENDEREÇO - CARGA: 1:N (Tem destino):**\
Este relacionamento é 1:N, pois um mesmo endereço pode ser destino de várias cargas diferentes, mas uma carga específica só possui um endereço de destino.
Além disso, toda carga necessariamente tem um endereço como destino, mas nem todo endereço precisa ser o destino de uma carga.



## Parte 2: Esquema Relacional

In [2]:
from IPython.display import Image, display

# URL da imagem
url = "https://i.postimg.cc/9MnMWH12/relacional-tp1-BDA-drawio.png"

# Exibir a imagem
display(Image(url=url))

## Parte 3: Álgebra Relacional

## Parte 4: SQL

### 4.1. Criação do Banco

In [9]:
import sqlite3

conn = sqlite3.connect("meubanco.db")
cursor = conn.cursor()

# cria a tabela
cursor.executescript("""
DROP TABLE IF EXISTS Endereco;
DROP TABLE IF EXISTS Cliente;
DROP TABLE IF EXISTS Produto;
DROP TABLE IF EXISTS Pedido;
DROP TABLE IF EXISTS Veiculo;
DROP TABLE IF EXISTS Motorista;
DROP TABLE IF EXISTS Rotas;
DROP TABLE IF EXISTS Carga;
DROP TABLE IF EXISTS Pedido_Produto;
DROP TABLE IF EXISTS Carga_Produto;
               
CREATE TABLE Endereco (
    id_endereco   INTEGER PRIMARY KEY,
    nome_rua      TEXT,
    num_rua       INTEGER,
    complemento   TEXT,
    cep           INTEGER,
    bairro        TEXT
);

CREATE TABLE Cliente (
    cpf_cliente   INTEGER PRIMARY KEY,
    nome_cliente  TEXT,
    id_endereco   INTEGER,
    FOREIGN KEY (id_endereco) REFERENCES Endereco(id_endereco)
);
               
CREATE TABLE Produto (
    id_produto    INTEGER PRIMARY KEY,
    nome_produto  TEXT,
    preco_produto    INTEGER
);
               
CREATE TABLE Pedido (
    id_pedido     INTEGER PRIMARY KEY,
    data_pedido   TEXT,
    total_pedido  INTEGER,
    entrega_prevista TEXT,
    cpf_cliente INTEGER,
    FOREIGN KEY (cpf_cliente) REFERENCES Cliente(cpf_cliente)
);
               
CREATE TABLE Veiculo (
    id_veiculo    INTEGER PRIMARY KEY,
    nome_veiculo  TEXT,
    tipo_veiculo  TEXT
);

CREATE TABLE Motorista (
    cpf_motorista INTEGER PRIMARY KEY,
    nome_motorista TEXT,
    id_veiculo    INTEGER,
    FOREIGN KEY (id_veiculo) REFERENCES Veiculo(id_veiculo)
);

CREATE TABLE Rotas (
    cpf_motorista TEXT,
    id_rota INTEGER,
    nome_rota TEXT,
    PRIMARY KEY (cpf_motorista, id_rota),   -- chave composta
    FOREIGN KEY (cpf_motorista) 
        REFERENCES Motorista(cpf_motorista) 
        ON DELETE CASCADE
);
               
CREATE TABLE Carga (
    id_carga            INTEGER PRIMARY KEY,
    id_endereco_origem  INTEGER,
    id_endereco_destino INTEGER,
    id_veiculo          INTEGER,
    status_carga        TEXT,
    data_entrega        TEXT,
    FOREIGN KEY (id_endereco_origem)  REFERENCES Endereco(id_endereco),
    FOREIGN KEY (id_endereco_destino) REFERENCES Endereco(id_endereco),
    FOREIGN KEY (id_veiculo)          REFERENCES Veiculo(id_veiculo)
);
               
CREATE TABLE Pedido_Produto (
    id_pedido     INTEGER,
    id_produto    INTEGER,
    qtd_pedida    INTEGER,
    total_produto INTEGER,
    PRIMARY KEY (id_pedido, id_produto),
    FOREIGN KEY (id_pedido)  REFERENCES Pedido(id_pedido),
    FOREIGN KEY (id_produto) REFERENCES Produto(id_produto)
);               

CREATE TABLE Carga_Produto (
    id_carga   INTEGER,
    id_produto INTEGER,
    tam_carga  INTEGER,
    PRIMARY KEY (id_carga, id_produto),
    FOREIGN KEY (id_carga)   REFERENCES Carga(id_carga),
    FOREIGN KEY (id_produto) REFERENCES Produto(id_produto)
);
""")

conn.commit()


### 4.2. Inserção de Dados